In [ ]:
import numpy as np
import argparse
import cv2
from imutils.video import VideoStream
from imutils.video import FPS
import time
import imutils
from scipy.spatial import distance
from typing import NamedTuple

In [ ]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [ ]:
net = cv2.dnn.readNetFromCaffe('SingleShotDetector/MobileNetSSD_deploy.prototxt.txt', 'SingleShotDetector/MobileNetSSD_deploy.caffemodel')

In [ ]:
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
fps = FPS().start()

In [ ]:
class Object(NamedTuple):
    name: str
    centroidx: float
    centroidy: float
    startX: float
    startY: float
    endX: float
    endY: float

In [ ]:
old_objects = []
while True:
    
    frame = vs.read()

    frame = imutils.resize(frame, width=400)

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    net.setInput(blob)
    detections = net.forward()
    
    new_objects = []

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):

        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.8:
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            
            x_centre = (box.astype("int")[0] + box.astype("int")[2]/2).astype("int")
            y_centre = (box.astype("int")[1] + box.astype("int")[3]/2).astype("int")   

            if y_centre >= 300:
                y_centre = 299
            if x_centre >= 400:
                x_centre = 399
       
            new_objects.append(Object(CLASSES[idx], x_centre, y_centre, startX, startY, endX, endY))
            
    if len(old_objects) == 0 :
        old_objects = new_objects
    else:
        if len(new_objects) > 0:
            new_centroids = []
            for x in new_objects:
                new_centroids.append([x[1],x[2]])

            old_centroids = []
            for x in old_objects:
                old_centroids.append([x[1],x[2]])

            dist = distance.cdist(np.asarray(old_centroids), np.asarray(new_centroids))

            old_idx = dist.min(axis=1).argsort()
            new_idx = dist.argmin(axis=1)[old_idx]


            for (row, col) in zip(old_idx, new_idx):
                old_objects[row] = new_objects[col]

    
    for obj in range(len(new_objects)):
        # display the prediction
        label = "{},{}".format(new_objects[obj][0], obj)
        cv2.rectangle(frame, (new_objects[obj][3], new_objects[obj][4]), (new_objects[obj][5], new_objects[obj][6]),
            COLORS[obj], 2)
        y = new_objects[obj][4] - 15 if new_objects[obj][4] - 15 > 15 else new_objects[obj][4] + 15
        cv2.putText(frame, label, (new_objects[obj][3], y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    # update the FPS counter
    fps.update()
    
    
    new_centroid = [] 

    
# stop the timer and display FPS information
fps.stop()
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

In [ ]:
new_objects

In [ ]:
old_objects

In [ ]:
new_centroids = []
for x in new_objects:
    new_centroids.append([x[1],x[2]])

In [ ]:
old_centroids = []
for x in old_objects:
    old_centroids.append([x[1],x[2]])

In [ ]:
np.asarray(old_centroids)

In [ ]:
np.asarray(new_centroids)

In [ ]:
D = distance.cdist(np.asarray(old_centroids), np.asarray(new_centroids))

In [ ]:
rows = D.min(axis=1).argsort()

# next, we perform a similar process on the columns by
# finding the smallest value in each column and then
# sorting using the previously computed row index list
cols = D.argmin(axis=1)[rows]

In [ ]:
for (row, col) in zip(rows, cols):
    print(old_objects[row])
    print(new_objects[col])

In [ ]:
rows[0]

In [ ]:
D

In [ ]:
cols[0]